In [ ]:
!wget https://github.com/git-lfs/git-lfs/releases/download/v3.4.0/git-lfs-linux-amd64-v3.4.0.tar.gz
!tar -xvzf git-lfs-linux-amd64-v3.4.0.tar.gz
!PREFIX=/opt/app-root/src/.local ./git-lfs-3.4.0/install.sh 

In [1]:
!git clone https://github.com/caikit/caikit-nlp.git

Cloning into 'caikit-nlp'...
remote: Enumerating objects: 5197, done.
remote: Counting objects: 100% (2355/2355), done.
remote: Compressing objects: 100% (616/616), done.
remote: Total 5197 (delta 1953), reused 1761 (delta 1736), pack-reused 2842
Receiving objects: 100% (5197/5197), 2.37 MiB | 42.49 MiB/s, done.
Resolving deltas: 100% (3819/3819), done.


In [9]:
!pip install ./caikit-nlp

Processing ./caikit-nlp
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for caikit-nlp: filename=caikit_nlp-0.4.1-py3-none-any.whl size=100780 sha256=6f2f5a0be04e24ee06a4540d4fafdcc9f06caac37332aa0a5169fae6f83f8a2d
  Stored in directory: /tmp/pip-ephem-wheel-cache-gbeh4bm3/wheels/16/ad/c7/0a0e6c2ffb12b0146d108e18055f8a4cdb030bdd038aad85d7
Successfully built caikit-nlp
  Attempting uninstall: caikit-nlp
    Found existing installation: caikit-nlp 0.4.1
    Uninstalling caikit-nlp-0.4.1:
      Successfully uninstalled caikit-nlp-0.4.1

[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!git clone https://github.com/opendatahub-io/caikit-tgis-serving.git


In [ ]:
!cp caikit-tgis-serving/utils/convert.py . 


In [2]:
!pip install --upgrade huggingface_hub
!huggingface-cli login --token $HUGGINGFACE_TOKEN


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /opt/app-root/src/.cache/huggingface/token
Login successful


In [1]:
!git clone https://huggingface.co/google/flan-t5-small

fatal: destination path 'flan-t5-small' already exists and is not an empty directory.


In [10]:
!./convert.py --model-path ./flan-t5-small/ --model-save-path ./flan-t5-small-caikit

<function register_backend_type at 0x7f86bf52b940> is still in the BETA phase and subject to change!


In [11]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)


In [12]:
OUTPUT_DIR = "flan-t5-small-caikit"
OUTPUT_DIR

'flan-t5-small-caikit'

In [13]:
upload_directory(OUTPUT_DIR, f"demo-models/{OUTPUT_DIR}")

flan-t5-small-caikit/config.yml -> demo-models/flan-t5-small-caikit/config.yml
flan-t5-small-caikit/artifacts/generation_config.json -> demo-models/flan-t5-small-caikit/artifacts/generation_config.json
flan-t5-small-caikit/artifacts/model.safetensors -> demo-models/flan-t5-small-caikit/artifacts/model.safetensors
flan-t5-small-caikit/artifacts/tokenizer.json -> demo-models/flan-t5-small-caikit/artifacts/tokenizer.json
flan-t5-small-caikit/artifacts/config.json -> demo-models/flan-t5-small-caikit/artifacts/config.json
flan-t5-small-caikit/artifacts/special_tokens_map.json -> demo-models/flan-t5-small-caikit/artifacts/special_tokens_map.json
flan-t5-small-caikit/artifacts/tokenizer_config.json -> demo-models/flan-t5-small-caikit/artifacts/tokenizer_config.json


In [14]:
list_objects(f"demo-models/")

demo-models/Llama-2-7b-hf-caikit/artifacts/config.json
demo-models/Llama-2-7b-hf-caikit/artifacts/generation_config.json
demo-models/Llama-2-7b-hf-caikit/artifacts/model-00001-of-00006.safetensors
demo-models/Llama-2-7b-hf-caikit/artifacts/model-00002-of-00006.safetensors
demo-models/Llama-2-7b-hf-caikit/artifacts/model-00003-of-00006.safetensors
demo-models/Llama-2-7b-hf-caikit/artifacts/model-00004-of-00006.safetensors
demo-models/Llama-2-7b-hf-caikit/artifacts/model-00005-of-00006.safetensors
demo-models/Llama-2-7b-hf-caikit/artifacts/model-00006-of-00006.safetensors
demo-models/Llama-2-7b-hf-caikit/artifacts/model.safetensors.index.json
demo-models/Llama-2-7b-hf-caikit/artifacts/special_tokens_map.json
demo-models/Llama-2-7b-hf-caikit/artifacts/tokenizer.json
demo-models/Llama-2-7b-hf-caikit/artifacts/tokenizer_config.json
demo-models/Llama-2-7b-hf-caikit/config.yml
demo-models/flan-t5-small-caikit/artifacts/config.json
demo-models/flan-t5-small-caikit/artifacts/generation_config.j